In [1]:
import os
import requests
import pandas as pd
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import gamma
from scipy.optimize import minimize
from scipy.interpolate import RectBivariateSpline
import emcee
import corner
import scipy.io as sio
from ipywidgets import FloatProgress
from IPython.display import display
import time
import os.path
from os import path

import sys
sys.path.insert(0, '../completenessContours')
import compute_num_completeness_w_ve_insol as kp

sys.path.insert(0, '..')
import occRateUtils as ut

import rateModels3D as rm3D

In [2]:
figDir = "summaryFigures"
stellarCatalog = "../stellarCatalogs/dr25_stellar_berger2020_clean_hab2.txt"
allStars = pd.read_csv(stellarCatalog)
# allStars = pd.read_csv("../stellarCatalogs/dr25_stellar_supp_gaia_clean_GKM.txt")
print("allStars has " + str(len(allStars)) + " stars")


allStars has 68885 stars


In [3]:
teffRange = (3900, 6300)

cs = rm3D.compSpace(periodName = "Insolation", 
               periodUnits = "Earth Flux",
               periodRange = (2.2, 0.2), 
               nPeriod = 61, 
               radiusName = "Radius", 
               radiusUnits = "$R_\oplus$",
               rpRange = (0.5, 2.5), 
               nRp = 61,
               tempName = "$T_\mathrm{eff}$", 
               tempUnits = "$R_\oplus$",
               tempRange = teffRange, 
               nTemp = 10)

model = rm3D.triplePowerLawTeffAvg(cs) # model 1

postZero = np.load("computeOccurrencefixedTeff_dr25_hab2_triplePowerLawTeffAvg_extrap_zero_uncertainty_out/occurenceRatePosteriors.npy")           
postConst= np.load("computeOccurrencefixedTeff_dr25_hab2_triplePowerLawTeffAvg_extrap_const_uncertainty_out/occurenceRatePosteriors.npy")           
medThetaZero = np.median(postZero, 0)
medThetaConst = np.median(postConst, 0)

In [4]:
-(4/3)*model.rateModel(1, 1, np.atleast_1d(4780), cs.periodRange, cs.rpRange, cs.tempRange, medThetaZero)

array([0.40657862])

In [5]:
-(4/3)*model.rateModel(1, 1, np.atleast_1d(4780), cs.periodRange, cs.rpRange, cs.tempRange, medThetaConst)

array([0.36564553])

In [6]:
ut.printMedianAndErrorbars(-(4/3)*model.rateModel(1, 1, np.atleast_1d(4780), cs.periodRange, cs.rpRange, cs.tempRange, postZero))

'0.384^{+0.267}_{-0.177}'

In [7]:
ut.printMedianAndErrorbars(-(4/3)*model.rateModel(1, 1, np.atleast_1d(4780), cs.periodRange, cs.rpRange, cs.tempRange, postConst))

'0.348^{+0.242}_{-0.161}'

In [14]:
def compute_Gamma_Earth(post):
    Samples = 1000
    GammaEarthDist = np.zeros((len(allStars), nSamples))
    f = FloatProgress(min=0, max=len(allStars))
    display(f)
    for i, s in allStars.iterrows():
        insol = s.radius**2 * (s.teff/4778)**4 / s.mass**(2./3)
    #     lmb = model.rateModel(insol, 1, np.atleast_1d(s.teff), cs.periodRange, cs.rpRange, cs.tempRange, medThetaZero)
        sampleIndex = np.floor(post.shape[0]*np.random.rand(nSamples)).astype(int)
        lmb = model.rateModel(insol, 1, np.atleast_1d(s.teff), cs.periodRange, cs.rpRange, cs.tempRange, post[sampleIndex,:])
        GammaEarthDist[i,:] = -(4/3)*insol*lmb
        f.value += 1

    return GammaEarthDist

In [18]:
GammaEarthZero = compute_Gamma_Earth(postZero)
ut.printMedianAndErrorbars(GammaEarthZero.flatten(), precision=2)

FloatProgress(value=0.0, max=68885.0)

'0.50^{+0.46}_{-0.26}'

In [17]:
GammaEarthConst = compute_Gamma_Earth(postConst)
ut.printMedianAndErrorbars(GammaEarthConst.flatten(), precision=2)

FloatProgress(value=0.0, max=68885.0)

'0.45^{+0.45}_{-0.24}'